In [23]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
from collections import OrderedDict
from pprint import pprint
import pandas as pd
import os

In [24]:
inputExcelFile = 'raw_cate.xlsx'
if os.path.isfile(inputExcelFile):
    xl = pd.ExcelFile(inputExcelFile)
else:
    print("[ERROR] Failed to load Excel File : %s" % (inputExcelFile))
    
df = xl.parse(''.join('raw'))
df[:10]

,1st,name,Link,API,Label,C_id,Total,Page_total
1,(NEW!) Meat & Seafood,Beef,https://thrivemarket.com/c/beef,NaN,Beef,1010,8,1
2,(NEW!) Meat & Seafood,Pork,https://thrivemarket.com/c/pork,NaN,Pork,1011,6,1
3,(NEW!) Meat & Seafood,Poultry,https://thrivemarket.com/c/poultry,NaN,Poultry,1012,6,1
4,(NEW!) Meat & Seafood,Seafood,https://thrivemarket.com/c/seafood,NaN,Seafood,1013,4,1
5,(NEW!) Meat & Seafood,Variety Packs,https://thrivemarket.com/c/variety-packs,NaN,Variety Packs,1016,8,1
6,Thrive Market Brand,Food,https://thrivemarket.com/c/thrive-food,NaN,Food,860,203,2
7,Thrive Market Brand,"Beauty, Bath & Body",https://thrivemarket.com/c/thrive-beauty-bath-...,NaN,"Beauty, Bath & Body",862,19,1
8,Thrive Market Brand,Health,https://thrivemarket.com/c/thrive-health,NaN,Health,863,81,1
9,Thrive Market Brand,Home,https://thrivemarket.com/c/thrive-home,NaN,Home,864,13,1
10,Thrive Market Brand,Babies & Kids,https://thrivemarket.com/c/thrive-babies-kids,NaN,Babies & Kids,865,8,1


In [25]:
C_id = df['C_id'].tolist()[-9:]
page_range =[int(i) for i in df['Page_total'].tolist()]
C_id 

[252, 261, 266, 267, 269, 258, 270, 271, 260]

In [26]:
df_product = pd.DataFrame(columns=('category', 'c_id','manufacturer', 'title', 'short_description', 'review_score', 'review_count', 'msrp', 'price', 'url'))
all_url = []

# C_id와 page_range에 맞춰서 url 만들어주기
for i,v in enumerate(C_id):
    url_base = 'https://api.thrivemarket.com/products?filter[categories]='
    c_id = str(v) # 루프
    for k in range(1, page_range[i] + 1):
        p_num = str(k) # 루프
        api_url = url_base + c_id + '&cur_page=' + p_num + '&page_size=120'
        all_url.append(api_url)

In [27]:
#api url 넣으면 원하는 값들 뽑아주기
for u in all_url:
    req_api_url = requests.get(u)
    bs_api_url = BeautifulSoup(req_api_url.content, "lxml").find('p')
    text_api_url = bs_api_url.text
    print('TRY CHECK URL:', all_url.index(u), u)
    try:
        data = json.loads(text_api_url, object_pairs_hook=OrderedDict)
        print('OK URL:', all_url.index(u), u)
    except:
        print('  Err  !')
        pass
    # data_2 = json.loads(text_api_url)
    want_to_get = [ 'manufacturer', 'title', 'short_description', 'review_score', 'review_count', 'msrp', 'price', 'url']
    data['applied_filters']['categories'][0]['label']
    for k in range(len(data['products'])):
        a = [data['applied_filters']['categories'][0]['label']]
        b = [data['applied_filters']['categories'][0]['key']]
        c = a + b + [ data['products'][k][i] for i in want_to_get]
        df_product.loc[len(df_product.index.tolist()) + 1] = c
df_product

TRY CHECK URL: 0 https://api.thrivemarket.com/products?filter[categories]=252&cur_page=1&page_size=120
  Err  !
TRY CHECK URL: 1 https://api.thrivemarket.com/products?filter[categories]=261&cur_page=1&page_size=120
OK URL: 1 https://api.thrivemarket.com/products?filter[categories]=261&cur_page=1&page_size=120
TRY CHECK URL: 2 https://api.thrivemarket.com/products?filter[categories]=266&cur_page=1&page_size=120
OK URL: 2 https://api.thrivemarket.com/products?filter[categories]=266&cur_page=1&page_size=120
TRY CHECK URL: 3 https://api.thrivemarket.com/products?filter[categories]=267&cur_page=1&page_size=120
OK URL: 3 https://api.thrivemarket.com/products?filter[categories]=267&cur_page=1&page_size=120
TRY CHECK URL: 4 https://api.thrivemarket.com/products?filter[categories]=269&cur_page=1&page_size=120
OK URL: 4 https://api.thrivemarket.com/products?filter[categories]=269&cur_page=1&page_size=120
TRY CHECK URL: 5 https://api.thrivemarket.com/products?filter[categories]=258&cur_page=1&pag

,category,c_id,manufacturer,title,short_description,review_score,review_count,msrp,price,url
1,Cat Treats,260,Halo,Liv-a-Littles Salmon Protein Treats,2.2 oz container,4.76,17,10.71,7.79,https://thrivemarket.com/p/halo-liv-a-littles-...
2,Cat Treats,260,Castor & Pollux,Organix Cat Treats,2 oz bag,4.52,23,4.49,3.29,https://thrivemarket.com/p/castor-pollux-organ...
3,Cat Treats,260,Halo,Liv-a-Littles Chicken Protein Treats,2.2 oz container,4.67,15,10.71,7.79,https://thrivemarket.com/p/halo-liv-a-littles-...
4,Cat Treats,260,Blue Buffalo,Kitty Yums Tempting Tuna Recipe Cat Treats,2 oz bag,5.00,4,3.99,2.99,https://thrivemarket.com/p/blue-buffalo-kitty-...
5,Cat Treats,260,Blue Buffalo,Wilderness Chicken & Duck Grain-Free Cat Treats,2 oz bag,5.00,3,3.99,3.29,https://thrivemarket.com/p/blue-buffalo-wilder...
6,Cat Accessories,261,Beco Pet,Natural Cat BecoBowl,1 each,4.17,12,3.99,3.29,https://thrivemarket.com/p/beco-pet-natural-ca...
7,Cat Accessories,261,Ore Originals,"Paw Pet Food Can Cover Set, Green",2 each,5.00,5,11.95,9.99,https://thrivemarket.com/p/ore-originals-pet-f...
8,Cat Accessories,261,Beco Pet,"Plush Cat Toy, Fish",1 each,4.67,6,6.49,5.29,https://thrivemarket.com/p/beco-pet-plush-cat-...
9,Cat Accessories,261,Ore Originals,"Black Cat Face Rubber Placemat, Recycled",1 each,4.33,6,12.95,8.99,https://thrivemarket.com/p/ore-originals-black...
10,Cat Accessories,261,Honest Pet,Eco Kitty Catcher,1 each,4.75,4,13.99,9.99,https://thrivemarket.com/p/honest-pet-eco-kitt...


In [31]:
writer = pd.ExcelWriter('[CATE]_products_list.xlsx')
df_product.to_excel(writer, 'products_list')
writer.save()